# Lab 1. PostgreSQL
Задача:
Подтвердить или опровергнуть “миф” о том, что виртуальная таблица
(VIEW), созданная на SQL запросе, работает медленнее, чем запрос к
оригинальной таблице напрямую.

In [143]:
import psycopg2
### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


In [144]:
### Создание cхемы
cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
connection.commit()

print("Схема 'ivan_patakin' создана.")

Схема 'ivan_patakin' создана.


In [145]:
### Создание таблицы
cursor.execute("CREATE TABLE ivan_patakin.student (id SERIAL PRIMARY KEY, name VARCHAR(50), age INT);")
connection.commit()

print("Таблица 'ivan_patakin.student' создана.")

Таблица 'ivan_patakin.student' создана.


In [146]:
### Заполнение таблицы данными
cursor.executemany(
    "INSERT INTO ivan_patakin.student (name, age) VALUES (%s, %s);",
    [(f"Student {i}", 20 + i % 10) for i in range(1, 100001)]
)
connection.commit()

print("Таблица 'ivan_patakin.student' заполнена данными.")

Таблица 'ivan_patakin.student' заполнена данными.


In [147]:
### Создание виртуальной таблицы
cursor.execute("CREATE OR REPLACE VIEW ivan_patakin.v_student AS SELECT * FROM ivan_patakin.student;")
connection.commit()

print("Виртуальная таблица 'ivan_patakin.v_student' создана.")

Виртуальная таблица 'ivan_patakin.v_student' создана.


In [148]:
### Создание запросов к виртуальной таблице и обычной таблице
import random

query_student = "SELECT * FROM ivan_patakin.student WHERE id = %s;"
query_v_student = "SELECT * FROM ivan_patakin.v_student WHERE id = %s;"

### Создание списка id для выборки
ids = [random.randint(1, 10000) for _ in range(1000)]

In [149]:
### Функция для выполнения запросов и получения времени выполнения
import time

def measure_query_time(query, params):
    start_time = time.time()
    cursor.execute(query, params)
    cursor.fetchall()
    return time.time() - start_time

In [150]:
### Измерение времени выполнения запросов
time_student = sum(measure_query_time(query_student, (i,)) for i in ids)
time_v_student = sum(measure_query_time(query_v_student, (i,)) for i in ids)

print(f"Время выполнения запросов к обычной таблице: {time_student:.4f} секунд")
print(f"Время выполнения запросов к виртуальной таблице: {time_v_student:.4f} секунд")
print(f"Разница во времени: {abs(time_student - time_v_student):.4f} секунд")

Время выполнения запросов к обычной таблице: 1.8876 секунд
Время выполнения запросов к виртуальной таблице: 1.4335 секунд
Разница во времени: 0.4541 секунд


In [151]:
### Выполнение 10 раз для обычной таблицы и виртуальной таблицы для определения разницы во времени
for i in range(10):
    ids = [random.randint(1, 100000) for _ in range(10000)]
    time_student = sum(measure_query_time(query_student, (i,)) for i in ids)
    time_v_student = sum(measure_query_time(query_v_student, (i,)) for i in ids)
    print(f"Время выполнения запросов к обычной таблице: {time_student:.4f} секунд")
    print(f"Время выполнения запросов к виртуальной таблице: {time_v_student:.4f} секунд")
    print(f"Разница во времени: {abs(time_student - time_v_student):.4f} секунд")

Время выполнения запросов к обычной таблице: 10.7179 секунд
Время выполнения запросов к виртуальной таблице: 15.6331 секунд
Разница во времени: 4.9152 секунд
Время выполнения запросов к обычной таблице: 14.7385 секунд
Время выполнения запросов к виртуальной таблице: 13.3874 секунд
Разница во времени: 1.3511 секунд
Время выполнения запросов к обычной таблице: 14.1886 секунд
Время выполнения запросов к виртуальной таблице: 10.9901 секунд
Разница во времени: 3.1986 секунд
Время выполнения запросов к обычной таблице: 12.4384 секунд
Время выполнения запросов к виртуальной таблице: 9.9980 секунд
Разница во времени: 2.4404 секунд
Время выполнения запросов к обычной таблице: 12.4977 секунд
Время выполнения запросов к виртуальной таблице: 10.4350 секунд
Разница во времени: 2.0628 секунд
Время выполнения запросов к обычной таблице: 10.3656 секунд
Время выполнения запросов к виртуальной таблице: 10.5313 секунд
Разница во времени: 0.1658 секунд
Время выполнения запросов к обычной таблице: 10.9288 

In [152]:
### Удаление таблицы и схемы
cursor.execute("DROP VIEW IF EXISTS ivan_patakin.v_student;")
cursor.execute("DROP TABLE IF EXISTS ivan_patakin.student;")
connection.commit()

print("Таблица удалена.")

Таблица и схема удалены.


In [ ]:
connection.rollback()

In [153]:
### Закрытие курсора и соединения
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
